In [ ]:
%pip install yfinance
%pip install pandas
%pip install numpy
%pip install matplotlib
%pip install statsmodels
%pip install datetime
%pip install scikit-learn
%pip install PyPortfolioOpt

In [ ]:
%pip install pandas-ta

## Load SP500 Data

In [4]:
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas_ta
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import warnings

In [5]:
sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
sp500['Symbol'] = sp500['Symbol'].str.replace('.', '-')
symbols_list = sp500['Symbol'].unique().tolist()

In [6]:
end_date = '2025-01-01'

start_date = pd.to_datetime(end_date) - pd.DateOffset(365*8)

In [7]:
df = yf.download(tickers=symbols_list, start=start_date, end=end_date, auto_adjust=False)

[*********************100%***********************]  503 of 503 completed
<ipython-input-7-387965520>:2: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df.stack()


Price               Adj Close       Close        High         Low        Open  \
Date       Ticker                                                               
2017-01-03 A        43.651138   46.490002   46.750000   45.740002   45.930000   
           AAPL     26.827246   29.037500   29.082500   28.690001   28.950001   
           ABBV     43.450203   62.410000   63.029999   61.939999   62.919998   
           ABT      33.456718   39.049999   39.080002   38.340000   38.630001   
           ACGL     27.224224   28.629999   29.143333   28.506666   28.943333   
...                       ...         ...         ...         ...         ...   
2024-12-31 XYL     115.293098  116.019997  117.110001  115.570000  116.779999   
           YUM     132.877213  134.160004  134.789993  133.250000  134.089996   
           ZBH     105.404167  105.629997  106.500000  104.959999  105.910004   
           ZBRA    386.220001  386.220001  387.410004  381.750000  383.420013   
           ZTS     161.892609  162.929993  163.750000  161.869995  162.899994   

Price                   Volume  
Date       Ticker               
2017-01-03 A         1739600.0  
           AAPL    115127600.0  
           ABBV      9328200.0  
           ABT       9677300.0  
           ACGL       942900.0  
...                        ...  
2024-12-31 XYL        641600.0  
           YUM       1217100.0  
           ZBH        683300.0  
           ZBRA       327900.0  
           ZTS       1327400.0  

[989160 rows x 6 columns]

In [ ]:
df = df.stack()

## Calculate features and technical indicators for each stock

In [9]:
from typing_extensions import dataclass_transform
# Garman-Klass Volatility
df['garman_klass_vol']=((np.log(df['High']) - np.log(df['Low']))**2)/2 - (2*np.log(2)-1)*((np.log(df['Adj Close']) - np.log(df['Open']))**2)

# RSI
df['rsi'] = df.groupby(level=1)['Adj Close'].transform(lambda x: pandas_ta.rsi(close=x, length=20))

# BBL
df['bb_low'] = df.groupby(level=1)['Adj Close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,0])
df['bb_mid'] = df.groupby(level=1)['Adj Close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,1])
df['bb_high'] = df.groupby(level=1)['Adj Close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,2])

In [10]:
# ATR
def compute_atr(stock_data):
  atr = pandas_ta.atr(high=stock_data['High'], low=stock_data['Low'], close=stock_data['Close'], length=14)

  return atr.sub(atr.mean()).div(atr.std())

df['atr'] = df.groupby(level=1, group_keys=False).apply(compute_atr)

In [11]:
# MACD
def compute_macd(close):
  macd = pandas_ta.macd(close=close, length=20).iloc[:,0]

  return macd.sub(macd.mean()).div(macd.std())

df['macd'] = df.groupby(level=1, group_keys=False)['Adj Close'].apply(compute_macd)

In [39]:
# Dollar Volume
df['dollar_volume'] = (df['Adj Close']*df['Volume'])/1e6

## Aggregate to montly level and filter top 150 most liquid stocks for each month

In [40]:
d_v = df.unstack('Ticker')['dollar_volume'].resample('ME').mean().stack('Ticker').to_frame('dollar_volume')

last_cols = [c for c in df.columns.unique(0) if c not in ['dollar_volume', 'Volume', 'Open', 'High', 'Low', 'Close']]

others = df.unstack()[last_cols].resample('ME').last().stack('Ticker', future_stack=True)

idf = pd.concat([d_v, others], axis=1).dropna()

Calculate 5-year rolling average of dollar volume for each stocks before filtering

In [41]:
idf['dollar_volume'] = idf['dollar_volume'].unstack('Ticker').rolling(5*12).mean().stack()

idf['dollar_vol_rank'] = idf.groupby('Date')['dollar_volume'].rank(ascending=False)

idf = idf[idf['dollar_vol_rank']<150].drop(['dollar_volume', 'dollar_vol_rank'], axis=1)

## Calculate Monthly Returns for different time horizons as features

In [ ]:
g = df.xs('AAPL', level=1)

outlier_cutoff = 0.005

lags = [1, 2, 3, 6, 9, 12]

for lag in lags:
  g[f'return_{lag}m'] = g['Adj Close'].pct_change(lag).pipe(lambda x: x.clip(lower=x.quantile(outlier_cutoff),
                                                                             upper=x.quantile(1-outlier_cutoff))).add(1).pow(1/lag).sub(1)